Методы решения бигармонических уравнений;
Метод вращений Гивенса

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sympy
from sympy import diff
import pandas as pd
import warnings
import random
from numpy import linalg as lg
import math as mh

In [5]:
#Variances:
L = 1 #Border
c0 = 0.4
eps = 10^(-12)
N = 4 
k = [5,10,20,40,80] #Number of sections

plt.figure(figsize=(16,50))


<Figure size 1152x3600 with 0 Axes>

<Figure size 1152x3600 with 0 Axes>

In [6]:
global colloc_mas #Colloc_points

def coll_init(N):
    colloc_mas = np.zeros([N+1]) 
    for m in range (N+1):
        colloc_mas[m] = np.cos((2*(m+1)-1)*np.pi/(2*(N+1)))
    colloc_mas = np.flipud(colloc_mas)
    print(colloc_mas)
    return colloc_mas

In [7]:
def Cheb(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    return v.subs(x,point)

def C_dx(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x)
    return v.subs(x,point)

def C_d2x(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x,2)
    return v.subs(x,point)   

def C_d3x(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x,3)
    return v.subs(x,point) 

def C_d4x(point, n):
    x = sympy.Symbol('x')
    v  = sympy.chebyshevt(n, x)
    v = diff(v,x,4)
    return v.subs(x,point) 

def Approx_C(c, n, N):
    bx = 0
    for t in range (N+1):
        x = sympy.Symbol('x')
        bx  = bx + c[t]*(sympy.chebyshevt(t, x).subs(x,n))
    #a =  bx.subs(x,n)
    return bx
    
def Approx_diff(c, n, N, m):
    bx = 0
    for t in range (N+1):
        x = sympy.Symbol('x')
        bx  = bx + c[t]*((diff(sympy.chebyshevt(t, x),x,m).subs(x,n)))
    #bxc = diff(bx,x,m)
    #a =  bxc.subs(x,n)                    
    return bx

In [8]:
def Householder(A_m, b, n_c, m_c):
    
    SVD = np.linalg.svd(A_m, compute_uv = False)
    Obs = max(SVD)/min(SVD)
    for i in range(n_c):
        v = np.copy(A_m[i:m_c, i])
        v[0] += np.linalg.norm(v)
        v.shape = (m_c-i, 1)
        b.shape = (m_c, 1)
        A_m[i:m_c, i:m_c] -= 2*np.dot(v, np.dot(v.T, np.copy(A_m[i:m_c, 
                                            i:m_c])))/np.dot(v.T, v)
        b[i:m_c] -= 2*np.dot(v, np.dot(v.T, np.copy(b[i:m_c])))/np.dot(v.T, v)
    QD = np.copy(A_m[0:n_c,0:n_c])
    B = np.copy(b[0:n_c])
    B.shape = (n_c, 1)
    Par = np.zeros(n_c)
    Par = Back_Gauss(np.copy(QD), n_c, np.copy(B))     
    return (Par, Obs)

In [9]:
def Coeff (A, Y):
    At = np.transpose(A)
    AH = np.dot(At,A)
    YH = np.dot(At,Y)
    AH_inv = lg.inv(AH)
    C = np.dot(AH_inv,YH)
    return C

In [10]:
def NU (N, C):    
    approx = np.zeros(M)
    for i in range(0,M):
        for n in range(0,N):
            approx[i] = approx[i] + C[n]*(data[i][0]**n)
    
    return approx

In [11]:
#Function
def Func(point):
    x = sympy.Symbol('x')
    w= x**2*(1-x)**2*sympy.exp(x)
    w = diff(w,x,4)
    return w.subs(x,point) 

#Accurate answer
def Acc_Func(point):
    x = sympy.Symbol('x')
    w  = x**2*(1-x)**2*sympy.exp(x)
    return w.subs(x,point) 

In [12]:
def Mtx(n,N,K, colloc_mas,c):
    
    A = np.zeros([(N+5),(N+1)])
    
    for i in range (N+1):
        for j in range (N+1):
            A[i][j] = C_d4x(colloc_mas[i],j)
    
    if n == 0:
        for j in range (N+1):
            A[N+1][j] = Cheb(1,j) + C_dx(1,j)
            A[N+2][j] = C_d2x(1,j) + C_d3x(1,j)
            A[N+3][j] = Cheb(-1,j)
            A[N+4][j] = C_dx(-1,j)
            
    elif n == K-1:
        for j in range (N+1):
            A[N+1][j] = Cheb(-1,j) - C_dx(-1,j)
            A[N+2][j] = C_d2x(-1,j) - C_d3x(-1,j)
            A[N+3][j] = Cheb(1,j)
            A[N+4][j] = C_dx(1,j)
            
    else:
        for j in range (N+1): 
            A[N+1][j] = Cheb(1,j) + C_dx(1,j)
            A[N+2][j] = C_d2x(1,j) + C_d3x(1,j)
            A[N+3][j] = Cheb(-1,j) - C_dx(-1,j)
            A[N+4][j] = C_d2x(-1,j) - C_d3x(-1,j)
   
    #A = np.fliplr(A)
    #A = np.flipud(A)
    print(A)
    return A
        

In [16]:
C = np.full((2*(N+5)),c0)
coll_mas1 = coll_init(N)
Mtx(0,N,2,coll_mas1,C)

[-9.51056516e-01 -5.87785252e-01  6.12323400e-17  5.87785252e-01
  9.51056516e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]


array([[  0.,   0.,   0.,   0., 192.],
       [  0.,   0.,   0.,   0., 192.],
       [  0.,   0.,   0.,   0., 192.],
       [  0.,   0.,   0.,   0., 192.],
       [  0.,   0.,   0.,   0., 192.],
       [  1.,   2.,   5.,  10.,  17.],
       [  0.,   0.,   4.,  48., 272.],
       [  1.,  -1.,   1.,  -1.,   1.],
       [  0.,   1.,  -4.,   9., -16.]])

In [491]:
def Coeff (A, Y):
    At = np.transpose(A)
    AH = np.dot(At,A)
    YH = np.dot(At,Y)
    AH_inv = lg.inv(AH)
    C = np.dot(AH_inv,YH)
    return C

def NU (N, C):    
    approx = np.zeros(M)
    for i in range(0,M):
        for n in range(0,N):
            approx[i] = approx[i] + C[n]*(data[i][0]**n)
    
    return approx

In [492]:
def b_vector(N,K,n,c, colloc_mas):
    b = np.zeros([N+5])
    bx = 0
    #c = np.zeros([N+1])
    
    #for l in range (N+1):
        #c[l] = c0
        
    for i in range(N+1):
        b[i] = Func(colloc_mas[i])
        
    if n == 0:
        for j in range (N+1):
            b[N+1] = 0
            b[N+2] = 0
            b[N+3] = Approx_C(c,1,N) - Approx_diff(c,1,N,1)
            b[N+4] = Approx_diff(c,1,N,2) - Approx_diff(c,1,N,3)
            
    elif n == K-1:
        for j in range (N+1):
            b[N+1] = 0
            b[N+2] = 0
            b[N+3] = Approx_C(c,-1,N) - Approx_diff(c,-1,N,1)
            b[N+4] = Approx_diff(c,-1,N,2) - Approx_diff(c,-1,N,3)
    else:
        for j in range (N+1): 
            b[N+1] = Approx_C(c,-1,N) - Approx_diff(c,-1,N,1)
            b[N+2] = Approx_diff(c,-1,N,2) - Approx_diff(c,-1,N,3)
            b[N+3] = Approx_C(c,1,N) - Approx_diff(c,1,N,1)
            b[N+4] = Approx_diff(c,1,N,2) - Approx_diff(c,1,N,3)
        
    return b

In [493]:
def Givens(A, Y):
    n = A.shape[1]
    M = Y.shape[0]
    print(M)
    
    AC = A.copy()
    YC = Y.copy()
     
    # Givens
    for i in range(0,n):
        for j in range(i + 1 , n+4):
            c = AC[i,i] / (AC[i,i]**2 + AC[j,i]**2) ** .5
            s = AC[j,i] / ((AC[i,i])**2 + (AC[j,i])**2) ** .5
            if ((AC[i,i])**2 + (AC[j,i])**2) == 0:
                c = 1
                s = 0
            print(c)
            tmp1 = AC[i,:] * c + AC[j,:] * s
            tmp2 = AC[i,:] * -s + AC[j,:] * c
            AC[i, :] = tmp1
            AC[j, :] = tmp2
            tmp3 = YC[i] * c + YC[j] * s
            tmp4 = YC[i] * (-s) + YC[j] * c
            YC[i] = tmp3
            YC[j] = tmp4
            
    
    # Gauss method_back
    
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        sum = 0
        for j in range(i+1, n):
            sum = sum + AC[i,j]*x[j]

        x[i] = (YC[i] - sum) / AC[i,i]
    return x


In [494]:
def Er(v, w):
    d = np.zeros(len(w))
    for i in range (len(w)):
        d[i] = np.abs(w[i] - v[i])
    return max(d)/max(w)

def Ea(w, v):
    d = np.zeros(len(w))
    for i in range (len(w)):
        d[i] = np.abs(w[i] - v[i])
    return max(d)

In [495]:
def Solution(N, K, points):
    C = np.full((K*(N+5)),c0)
    CP = np.zeros(K*(N+5))
    s = 0
    n = 0
    c1 = C[0:N+1]
    c2 = np.zeros((N+1))
    print(c2)

    a = 1
    
    while a==1:
        
            
        A = Mtx(n, N, K, points,c1)
        B = b_vector(N,K,n,c1,points)
        #c2 = Givens(A,B)
        
        c2 = Coeff(A,B)
        
        c1 = C[n+N+1:n + 2*(N+1)]
        print(c2)
        CP[n*(N+1):(n+1)*(N+1)] = c2
        n = n + 1
        
        #for i in range (K*(N+5)):
            #if max(CP[i] - C[i]) < eps:
                #break
                
        #for i in range (K*(N+5)):
        if max(CP - C) < eps:
                break
                
        if n > K:
            s = s + 1
            n = 0
            for l in range ((K*(N+5))):
                C[l] = CP[l]        
                
    B_Fin = A.CP
                
    return s, n, CP, B_Fin
        

In [496]:
print("Размер сетки, К | ||Ea|| | Порядок сходимости | ||Er|| | Порядок сходимости | |μ(A)")
print("--------------------------------------------------------------------------------")
plt.figure(figsize=(16,50))
for i in range (len(k)):
    K = k[i]
    h = L/(2*K)
    colloc_mas = coll_init(N)
    X_Int = np.zeros(K*(N+1))
    for j in range (K):
        X_Int[0 + j*(N+1):(j+1)*(N+1)] = colloc_mas
        
        
    s, n, C_Fin, B_Fin = Solution(N,K,coll_dot)
    
    X = np.arange(0,1,100)
    B_Acc = Acc_Func(X)
    plt.subplot(5,1,i+1)
    plt.title("K = %d"%(K))
    plt.plot(x, y_sol, 'b', x, y_ac, 'g--')   
    
    if i == 0:
        ER_p = E_r(np.copy(y_sol),np.copy(y_ac))
        EA_p = E_a(np.copy(y_sol),np.copy(y_ac))
        print("%d | %e | %e | %e | %e | %f |"%(K_list[i], EA_p, 0 ,ER_p ,0 ,mu))
        print("--------------------------------------------------------------------------------")
    else:
        ER_n = E_r(np.copy(y_sol),np.copy(y_ac))
        EA_n = E_a(np.copy(y_sol),np.copy(y_ac))        
        print("%d | %e | %e | %e | %e | %f |"%(K_list[i], E_a(y_sol,y_ac), np.log2(EA_p/EA_n) ,
                                               E_r(y_sol,y_ac), np.log2(ER_p/ER_n),mu))
        ER_p = E_r(np.copy(y_sol),np.copy(y_ac))
        EA_p = E_a(np.copy(y_sol),np.copy(y_ac))
        print("--------------------------------------------------------------------------------")
plt.show()


Размер сетки, К | ||Ea|| | Порядок сходимости | ||Er|| | Порядок сходимости | |μ(A)
--------------------------------------------------------------------------------
[-9.51056516e-01 -5.87785252e-01  6.12323400e-17  5.87785252e-01
  9.51056516e-01]
[0. 0. 0. 0. 0.]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
[-17.37069769   0.67006055   6.01866555  -1.79401211   0.22808059]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[40.87002897 -3.4        -9.50947978  1.4         0.22808059]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.

[ 9.81590713e+02 -8.66547451e+01 -7.08673037e+01  6.57701463e-01
  2.28080585e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[ 3.85020295e+09  4.66691288e+08 -7.77818779e+08 -1.55563753e+08
  2.28080587e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[-3.50019887e+08 -2.12134157e+07  1.39064846e+08  5.89315597e+05
  2.28080585e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]
 [  1.   1.   1.   1.   1.]
 [  0.   1.   4.   9.

[-7.85304583e+16  3.73630163e+16  3.67815578e+16  3.06512982e+15
  2.03125000e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[ 6.45282849e+15  6.60377185e+14 -1.96226364e+15  2.26415035e+14
  2.26562500e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9. -16.]]
[ 4.53402638e+17 -2.48703309e+17  1.05609551e+16 -8.80079594e+14
  2.34375000e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48.

  1.04857600e+06]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[ 7.66502261e+03 -2.57933055e+02 -2.02440054e+02  6.08118240e-01
  2.28080585e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[-3.51722680e+27 -4.26330522e+26  7.10550869e+26  1.42110174e+26
 -4.29496730e+09]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[ 3.19747891e+26  1.93786601e+25 -1.27037883e+26 -5.38296

[ 3.07148549e+34  1.86150635e+33 -1.22032083e+34 -5.17085098e+31
  3.60287970e+16]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]
 [  1.   1.   1.   1.   1.]
 [  0.   1.   4.   9.  16.]]
[ 7.17389281e+34 -3.41317598e+34 -3.36005875e+34 -2.80004895e+33
  2.47697980e+16]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[-5.89477012e+33 -6.03265948e+32  1.79256167e+33 -2.06834039e+32
 -9.00719925e+15]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9.

[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[ 2.06518733e+04 -4.33123968e+02 -3.33814083e+02  6.08215082e-01
  2.28080585e-01]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[ 3.21304735e+45  3.89460285e+44 -6.49100474e+44 -1.29820095e+44
  2.47588008e+27]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[-2.92095213e+44 -1.77027402e+43  1.16051297e+44  4.91742784e+41
 -6.189700

[-2.80585497e+52 -1.70051816e+51  1.11478413e+52  4.72366156e+49
 -4.15383749e+34]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]
 [  1.   1.   1.   1.   1.]
 [  0.   1.   4.   9.  16.]]
[-6.55347482e+52  3.11799513e+52  3.06947162e+52  2.55789301e+51
 -1.29807421e+34]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -2.   5. -10.  17.]
 [  0.   0.   4. -48. 272.]]
[ 5.38497418e+51  5.51093849e+50 -1.63753601e+51  1.88946462e+50
  0.00000000e+00]
[[  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  0.   0.   0.   0. 192.]
 [  1.   2.   5.  10.  17.]
 [  0.   0.   4.  48. 272.]
 [  1.  -1.   1.  -1.   1.]
 [  0.   1.  -4.   9.

KeyboardInterrupt: 

<Figure size 1152x3600 with 0 Axes>

In [ ]:
d = {"Ea":np.zeros(5),"OoC":np.zeros(5),"Er":np.zeros(5),"OoC":np.zeros(5),"N_iter":np.zeros(5),"mu(Ab)":np.zeros(5),"mu(Ai)":np.zeros(5)}
df = pd.DataFrame(d, index=['5', '10', '20','40','80'])

df.iat[N-1,0] = np.linalg.cond(np.dot(list[0].transpose(), list[0]))
df.iat[N-1,1] = SME(Y_int)
    
    
df.iat[N-1,2] = lg.cond(list[0])
df.iat[N-1,3] = SME(Y_Giv)

In [111]:
a = [1, 3, 8, 7]
a[1:3] = [0, 0]
print(a)

[1, 0, 0, 7]
